<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%EC%B5%9C%EC%A2%85code/(%EC%A7%84%EC%A7%9C%EC%B5%9C%EC%A2%85)%20%EC%9A%94%EC%95%BD%EB%AC%B8_%EC%A6%9D%EA%B0%95_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [1]:
pip install transformers sentencepiece torch rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=620cf41342b4fa6fe931ff697826970fffde5dee3df11e161b12b2c37fb79a53
  S

In [2]:
!pip install konlpy
!pip install textrankr
!pip install JPype1>=0.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 26.2 MB/s eta 0:00:00


# 요약문 알고리즘

## ① TextRank

In [3]:
import re
import torch
import numpy as np
#import kss
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertForSequenceClassification
from rouge_score import rouge_scorer
import re
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def textrank_summary(text, ratio=0.2):
    if not isinstance(text, str) or text.strip() == "":
        return ""  # Return empty string if input is invalid
    # 문장 분리 (". "만으로 나누면 오류 발생 가능 → 정규식 사용)
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    # 빈 문장 필터링
    sentences = [s.strip() for s in sentences if s.strip()]
    if len(sentences) == 0:
        return ""  # No valid sentences

    # 최소 한 문장만 포함되도록 설정
    top_n = max(1, int(len(sentences) * ratio))

    # TF-IDF 벡터화 (stopwords 오류 방지)
    vectorizer = TfidfVectorizer()
    try:
        sentence_vectors = vectorizer.fit_transform(sentences)
    except ValueError:  # 빈 vocabulary 오류 발생 시 원문 반환
        return text

    # 문장 유사도 계산
    similarity_matrix = cosine_similarity(sentence_vectors, sentence_vectors)

    # 그래프 생성 및 PageRank 적용
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)

    # 중요 문장 정렬 후 선택
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summarized_sentences = [ranked_sentences[i][1] for i in range(top_n)]

    return ' '.join(summarized_sentences)

## ② KoBART

In [4]:
!git clone https://github.com/seujung/KoBART-summarization.git

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 155 (delta 49), reused 44 (delta 40), pack-reused 83 (from 1)
Receiving objects: 100% (155/155), 37.24 MiB | 26.82 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [5]:
!pip install loguru
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 28.4 MB/s eta 0:00:00


In [6]:
cd /content/drive/MyDrive/KoBART-summarization

[Errno 2] No such file or directory: '/content/drive/MyDrive/KoBART-summarization'
/content


In [7]:
!pip install -r requirements.txt
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu101 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1

In [8]:
# 클론한 파일 제공 데이터
%cd /content/KoBART-summarization/data
!tar -zxvf train.tar.gz
!tar -zxvf test.tar.gz
!ls

/content/KoBART-summarization/data
train.tsv
test.tsv
test.tar.gz  test.tsv  train.tar.gz  train.tsv


In [9]:
#Test
!pip install git+https://github.com/SKT-AI/KoBART

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-req-build-8zdnjhqt
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-req-build-8zdnjhqt
  Resolved https://github.com/SKT-AI/KoBART to commit eec563bfccf723cae8fd0fff02d5b2b09e847516
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of kobart to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from kobart) (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.7.1


In [10]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [11]:
# KoBART 기반 추상적 요약
def Kobart_summary(text):
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=200, num_beams=3, no_repeat_ngram_size=3)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

# 데이터 불러오기

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   NSMC_0.02_aug_mlm.pkl   NSMC_0.1_aug_mlm.pkl   이전   졸업논문


In [13]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_0.8_증분대상_1000개.csv', encoding='utf-8-sig')

In [14]:
df.head(3)

,Unnamed: 0,id,document,label
0,0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0
1,1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0
2,2,6549317,오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 고대루네~! 그냥 갖...,0


In [15]:
df_2=df[df['label']==0]

In [16]:
df_2.columns

Index(['Unnamed: 0', 'id', 'document', 'label'], dtype='object')

In [ ]:
df_2['summary_ext'] = df_2['document'].apply(textrank_summary)
df_2['summary_abs'] = df_2['document'].apply(Kobart_summary)

In [ ]:
df_2[['text', 'summary_ext', 'summary_abs']].head(10)

In [ ]:
df_2.to_csv('요약문증강 최종_0.8_1000개.csv', encoding='utf-8-sig')

# 요약문 평가
- ROUGE (Recall-Oriented Understudy for Gisting Evaluation) 점수는 자동 요약 및 자연어 생성 모델의 성능을 평가하는 대표적인 방법 중 하나입니다. 주요 ROUGE 지표는 다음과 같습니다.
- ROUGE-1: 요약된 문장에서 정답과 일치하는 단어 단위(n=1)의 개수를 기반으로 측정
- ROUGE-2: 정답과 요약이 공유하는 2-그램(n=2) 단위의 개수 기반 평가
- ROUGE-L: 정답과 요약된 문장 간 가장 긴 공통 부분 문자열(LCS, Longest Common Subsequence) 기반 평가

In [ ]:
df=df_2

In [ ]:
df.columns

Index(['demographic_category', 'demographic_group', 'context', 'context_en',
       'sentence', 'sentence_en', 'context_label', 'context_sub_label',
       'sentence_label', 'sentence_sub_label', 'annotated_demo', 'text',
       'summary_ext', 'summary_abs'],
      dtype='object')

In [ ]:
from rouge_score import rouge_scorer
import pandas as pd

# ROUGE 점수 계산 함수
def calculate_rouge(df, summary_col, reference_col):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, pred in zip(df[reference_col], df[summary_col]):
        score = scorer.score(ref, pred)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    return {metric: sum(values)/len(values) for metric, values in scores.items()}

# 평가 수행
rouge_scores_abs = calculate_rouge(df, 'summary_abs', 'text') #정답데이터 : summary (=text)
rouge_scores_ext = calculate_rouge(df, 'summary_ext', 'text') #정답데이터 : summary (=text)

# 결과 출력
print("ROUGE 점수 (KoBART 요약)", rouge_scores_abs)
print("ROUGE 점수 (TextRank 요약)", rouge_scores_ext)


ROUGE 점수 (KoBART 요약) {'rouge1': 0.15833333333333333, 'rouge2': 0.0, 'rougeL': 0.15833333333333333}
ROUGE 점수 (TextRank 요약) {'rouge1': 0.5, 'rouge2': 0.0, 'rougeL': 0.5}
